<a href="https://colab.research.google.com/github/jakless/mgr2024/blob/main/Model_20240427.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile

def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

# Przykładowe użycie funkcji:
zip_file_path = '/content/images.zip'
extract_to_path = '/content'
unzip_file(zip_file_path, extract_to_path)


1 Wersja - wykonuje się


In [4]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Funkcja do parsowania pliku XML
def parse_xml_annotation(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Inicjalizacja listy dla opisów obiektów
    annotations = []

    # Iteracja po każdym obiekcie w pliku XML
    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Dodanie opisu obiektu do listy
        annotations.append({'class_name': class_name,
                            'xmin': xmin,
                            'ymin': ymin,
                            'xmax': xmax,
                            'ymax': ymax})

    return annotations

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
valid_images = []
valid_labels = []

# Wczytanie danych obrazów i ich opisów dla danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        xml_file_path = os.path.join(train_dir, image_name.replace('.jpg', '.xml'))

        # Wczytanie obrazu
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Wczytanie opisów obiektów
        annotations = parse_xml_annotation(xml_file_path)

        # Przetworzenie opisów obiektów
        for annotation in annotations:
            class_name = annotation['class_name']
            xmin, ymin, xmax, ymax = annotation['xmin'], annotation['ymin'], annotation['xmax'], annotation['ymax']

            # Wyodrębnienie obszaru obiektu z obrazu
            object_image = image[ymin:ymax, xmin:xmax]

            # Zmiana rozmiaru obrazu na wymagany przez model EfficientNet
            object_image = cv2.resize(object_image, (224, 224))

            # Przetworzenie obrazu do formatu akceptowanego przez EfficientNet
            object_image = img_to_array(object_image)
            object_image = preprocess_input(object_image)

            # Dodanie obrazu i etykiety do listy danych treningowych
            train_images.append(object_image)
            train_labels.append(class_name)

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')
train_labels = np.array(train_labels)

# Wczytanie danych obrazów i ich opisów dla danych walidacyjnych
for image_name in os.listdir(valid_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(valid_dir, image_name)
        xml_file_path = os.path.join(valid_dir, image_name.replace('.jpg', '.xml'))

        # Wczytanie obrazu
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Wczytanie opisów obiektów
        annotations = parse_xml_annotation(xml_file_path)

        # Przetworzenie opisów obiektów
        for annotation in annotations:
            class_name = annotation['class_name']
            xmin, ymin, xmax, ymax = annotation['xmin'], annotation['ymin'], annotation['xmax'], annotation['ymax']

            # Wyodrębnienie obszaru obiektu z obrazu
            object_image = image[ymin:ymax, xmin:xmax]

            # Zmiana rozmiaru obrazu na wymagany przez model EfficientNet
            object_image = cv2.resize(object_image, (224, 224))

            # Przetworzenie obrazu do formatu akceptowanego przez EfficientNet
            object_image = img_to_array(object_image)
            object_image = preprocess_input(object_image)

            # Dodanie obrazu i etykiety do listy danych walidacyjnych
            valid_images.append(object_image)
            valid_labels.append(class_name)

# Konwersja list na tablice numpy
valid_images = np.array(valid_images, dtype='float32')
valid_labels = np.array(valid_labels)

# Przekonwertowanie etykiet na liczby całkowite
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)

# Wyświetlenie unikalnych wartości przypisanych do klas
print("Unikalne wartości przypisane do klas:", label_encoder.classes_)

# Budowa modelu EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie warstw do wyjścia modelu
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Tworzenie końcowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(valid_images, valid_labels_encoded))


Unikalne wartości przypisane do klas: ['BLUE' 'GREY']
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/10
25/25 [==============================] - 252s 9s/step - loss: 0.1343 - accuracy: 0.9550 - val_loss: 0.2814 - val_accuracy: 0.9500
Epoch 2/10
25/25 [==============================] - 236s 9s/step - loss: 0.0467 - accuracy: 0.9900 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 3/10
25/25 [==============================] - 225s 9s/step - loss: 0.0256 - accuracy: 0.9900 - val_loss: 3.5379 - val_accuracy: 0.5600
Epoch 4/10
25/25 [==============================] - 225s 9s/step - loss: 0.0094 - accuracy: 0.9950 - val_loss: 0.2307 - val_accuracy: 0.9550
Epoch 5/10
25/25 [==============================] - 212s 8s/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.2788 - val_accuracy: 0.9200
Epoch 6/10
25/25 [==============================] - 223s 9s/step - loss: 7.8682e-04 - accuracy: 1.0000 - val_loss: 0.4616 - val_accuracy: 0.8000
Epoch 7/10
25/25 [=============

2 Wersja z zapisem do h5

In [5]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Funkcja do parsowania pliku XML
def parse_xml_annotation(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Inicjalizacja listy dla opisów obiektów
    annotations = []

    # Iteracja po każdym obiekcie w pliku XML
    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Dodanie opisu obiektu do listy
        annotations.append({'class_name': class_name,
                            'xmin': xmin,
                            'ymin': ymin,
                            'xmax': xmax,
                            'ymax': ymax})

    return annotations

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
valid_images = []
valid_labels = []

# Wczytanie danych obrazów i ich opisów dla danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        xml_file_path = os.path.join(train_dir, image_name.replace('.jpg', '.xml'))

        # Wczytanie obrazu
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Wczytanie opisów obiektów
        annotations = parse_xml_annotation(xml_file_path)

        # Przetworzenie opisów obiektów
        for annotation in annotations:
            class_name = annotation['class_name']
            xmin, ymin, xmax, ymax = annotation['xmin'], annotation['ymin'], annotation['xmax'], annotation['ymax']

            # Wyodrębnienie obszaru obiektu z obrazu
            object_image = image[ymin:ymax, xmin:xmax]

            # Zmiana rozmiaru obrazu na wymagany przez model EfficientNet
            object_image = cv2.resize(object_image, (224, 224))

            # Przetworzenie obrazu do formatu akceptowanego przez EfficientNet
            object_image = img_to_array(object_image)
            object_image = preprocess_input(object_image)

            # Dodanie obrazu i etykiety do listy danych treningowych
            train_images.append(object_image)
            train_labels.append(class_name)

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')
train_labels = np.array(train_labels)

# Wczytanie danych obrazów i ich opisów dla danych walidacyjnych
for image_name in os.listdir(valid_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(valid_dir, image_name)
        xml_file_path = os.path.join(valid_dir, image_name.replace('.jpg', '.xml'))

        # Wczytanie obrazu
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Wczytanie opisów obiektów
        annotations = parse_xml_annotation(xml_file_path)

        # Przetworzenie opisów obiektów
        for annotation in annotations:
            class_name = annotation['class_name']
            xmin, ymin, xmax, ymax = annotation['xmin'], annotation['ymin'], annotation['xmax'], annotation['ymax']

            # Wyodrębnienie obszaru obiektu z obrazu
            object_image = image[ymin:ymax, xmin:xmax]

            # Zmiana rozmiaru obrazu na wymagany przez model EfficientNet
            object_image = cv2.resize(object_image, (224, 224))

            # Przetworzenie obrazu do formatu akceptowanego przez EfficientNet
            object_image = img_to_array(object_image)
            object_image = preprocess_input(object_image)

            # Dodanie obrazu i etykiety do listy danych walidacyjnych
            valid_images.append(object_image)
            valid_labels.append(class_name)

# Konwersja list na tablice numpy
valid_images = np.array(valid_images, dtype='float32')
valid_labels = np.array(valid_labels)

# Przekonwertowanie etykiet na liczby całkowite
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)

# Wyświetlenie unikalnych wartości przypisanych do klas
print("Unikalne wartości przypisane do klas:", label_encoder.classes_)

# Budowa modelu EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie warstw do wyjścia modelu
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Tworzenie końcowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_images, train_labels_encoded, epochs=1, batch_size=32, validation_data=(valid_images, valid_labels_encoded))

# Zapis modelu do pliku HDF5
model.save("efficientnet_model.h5")

# Zapis modelu do pliku Keras
model.save("efficientnet_model.keras")



Unikalne wartości przypisane do klas: ['BLUE' 'GREY']
25/25 [==============================] - 244s 9s/step - loss: 0.1692 - accuracy: 0.9150 - val_loss: 1.5061 - val_accuracy: 0.8100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
